In [22]:
from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        def inference(params_dict, *args):
            seed = params_dict['seed']
            seed = seed.lower()
            generated = seed
            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(seed):
                    x_pred[0, t, char_indices[char]] = 1.
                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, 0.5)
                next_char = indices_char[next_index]
                generated += next_char
                seed = seed[1:] + next_char
            return generated
        print(inference({'seed': sentence}))

        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=6,
          callbacks=[print_callback])

corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...
Epoch 1/1
200192/200285 [============================>.] - ETA: 0s - loss: 2.0397
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "the evil as a
benefit of which the uses "
the evil as a
benefit of which the uses and of a the present the warings the conceuse the such a the present and and by every the one the experitious and alour and completesm to a such the really not should the
oble waster of a the has to knowledge the inflation the indiss of for but the world who have a the which the hasse the
expect to a morality and a relight to hasse of himself to the hersely and a such a still and the soul in a con

----- diversity: 0.5
----- Generating with seed: "the evil as a
benefit of which the uses "
the evil as a
benefit of which the uses of the pays of the
sufferingly the his in in a goed the interple the himself all the self and soul the there acting of the c

In [2]:
import tensorflow as tf

In [9]:
tfjs.converters.save_keras_model(model,'/root')

In [38]:
import json
json.dumps(char_indices)

'{"0": 10, "w": 49, "l": 38, "o": 41, "5": 15, "6": 16, ")": 6, "h": 34, "8": 18, "t": 46, ".": 9, "k": 37, "d": 30, "!": 2, "\'": 4, "=": 22, "[": 24, "s": 45, ";": 21, "]": 25, "e": 31, ",": 7, "\\"": 3, "p": 42, "g": 33, "?": 23, " ": 1, ":": 20, "(": 5, "\\n": 0, "q": 43, "1": 11, "4": 14, "2": 12, "z": 52, "7": 17, "j": 36, "c": 29, "y": 51, "f": 32, "_": 26, "\\u00e9": 55, "b": 28, "a": 27, "\\u00e6": 54, "9": 19, "r": 44, "x": 50, "\\u00e4": 53, "v": 48, "n": 40, "i": 35, "u": 47, "-": 8, "\\u00eb": 56, "m": 39, "3": 13}'

In [39]:
json.dumps(indices_char)

'{"0": "\\n", "1": " ", "2": "!", "3": "\\"", "4": "\'", "5": "(", "6": ")", "7": ",", "8": "-", "9": ".", "10": "0", "11": "1", "12": "2", "13": "3", "14": "4", "15": "5", "16": "6", "17": "7", "18": "8", "19": "9", "20": ":", "21": ";", "22": "=", "23": "?", "24": "[", "25": "]", "26": "_", "27": "a", "28": "b", "29": "c", "30": "d", "31": "e", "32": "f", "33": "g", "34": "h", "35": "i", "36": "j", "37": "k", "38": "l", "39": "m", "40": "n", "41": "o", "42": "p", "43": "q", "44": "r", "45": "s", "46": "t", "47": "u", "48": "v", "49": "w", "50": "x", "51": "y", "52": "z", "53": "\\u00e4", "54": "\\u00e6", "55": "\\u00e9", "56": "\\u00eb"}'